# Day 3 - Conversational AI - aka Chatbot!

In [2]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyBw


In [4]:
# Initialize

openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [5]:
system_message = "You are a helpful assistant"

# Please read this! A change from the video:

In the video, I explain how we now need to write a function called:

`chat(message, history)`

Which expects to receive `history` in a particular format, which we need to map to the OpenAI format before we call OpenAI:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

But Gradio has been upgraded! Now it will pass in `history` in the exact OpenAI format, perfect for us to send straight to OpenAI.

So our work just got easier!

We will write a function `chat(message, history)` where:  
**message** is the prompt to use  
**history** is the past conversation, in OpenAI format  

We will combine the system message, history and latest message, then call OpenAI.

In [5]:
# Simpler than in my video - we can easily create this function that calls OpenAI
# It's now just 1 line of code to prepare the input to OpenAI!

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [6]:
# Definicja funkcji `chat`, która obsługuje rozmowę z modelem OpenAI
def chat(message, history):
    # Tworzymy listę `messages`, która zawiera:
    # 1. Wiadomość systemową (zdefiniowaną wcześniej jako `system_message`).
    # 2. Historię rozmowy (przekazaną do funkcji jako `history`).
    # 3. Nową wiadomość użytkownika (`message`).
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    # Wypisujemy historię rozmowy do konsoli (pomocne przy debugowaniu)
    print("History is:")
    print(history)

    # Wypisujemy pełną listę `messages` po dodaniu nowej wiadomości
    print("And messages is:")
    print(messages)

    # Wysyłamy przygotowaną listę `messages` do OpenAI,
    # używając modelu `MODEL` oraz włączając tryb strumieniowania `stream=True`.
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    # Tworzymy pusty string, który będzie stopniowo uzupełniany odpowiedzią modelu.
    response = ""
    
    # Iterujemy po kolejnych fragmentach odpowiedzi zwracanych przez OpenAI.
    for chunk in stream:
        # Każdy fragment (`chunk`) zawiera obiekt `choices`, z którego pobieramy tekst odpowiedzi.
        # Jeśli `delta.content` nie zawiera tekstu (może być `None`), dodajemy pusty string `''`.
        response += chunk.choices[0].delta.content or ''
        
        # Używamy `yield`, aby zwracać częściową odpowiedź na bieżąco.
        # Dzięki temu możemy wyświetlać wynik w czasie rzeczywistym.
        yield response

## And then enter Gradio's magic!

In [7]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


History is:
[{'role': 'user', 'metadata': None, 'content': 'cześć', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Cześć! Jak mogę Ci dzisiaj pomóc w naszym sklepie? Szukasz czegoś szczególnego?', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'szukam tanich telefonów', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Rozumiem, że szukasz telefonów, ale w naszym sklepie specjalizujemy się w odzieży i akcesoriach. Może zainteresuje Cię coś na naszych wyprzedażach? Mamy świetne oferty na kapelusze w wysokości 60% zniżki oraz na inne ubrania, które są przecenione o 50%! Jeśli chcesz, mogę pokazać Ci nasze najnowsze modele kapeluszy. Co o tym myślisz?', 'options': None}]
And messages is:
[{'role': 'system', 'content': "You are a helpful assistant in a clothes store. You should try to gently encourage the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. For example, if the customer says '

In [8]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

History is:
[]
And messages is:
[{'role': 'system', 'content': "You are a helpful assistant in a clothes store. You should try to gently encourage the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. For example, if the customer says 'I'm looking to buy a hat', you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'Encourage the customer to buy hats if they are unsure what to get."}, {'role': 'user', 'content': 'cześć'}]
History is:
[{'role': 'user', 'metadata': None, 'content': 'cześć', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Cześć! Jak mogę Ci dzisiaj pomóc w naszym sklepie? Szukasz czegoś szczególnego?', 'options': None}]
And messages is:
[{'role': 'system', 'content': "You are a helpful assistant in a clothes store. You should try to gently encourage the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. F

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
# Funkcja `chat` obsługuje komunikację z modelem językowym OpenAI, umożliwiając interaktywną rozmowę.
def chat(message, history):
    # Tworzymy listę `messages`, która zawiera:
    # 1. Wiadomość systemową (określa kontekst rozmowy i zachowanie modelu).
    # 2. Historię wcześniejszych wiadomości użytkownika i asystenta.
    # 3. Nową wiadomość użytkownika.
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    # Wysyłamy zapytanie do OpenAI, przekazując:
    # - `model=MODEL` – określa, który model OpenAI zostanie użyty.
    # - `messages=messages` – przekazujemy pełen kontekst rozmowy.
    # - `stream=True` – aktywujemy strumieniowanie, co pozwala na stopniowe wyświetlanie odpowiedzi.
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    # Inicjalizujemy pusty string `response`, który będzie przechowywał wygenerowaną odpowiedź.
    response = ""

    # Iterujemy po kolejnych fragmentach odpowiedzi zwracanych przez OpenAI.
    for chunk in stream:
        # Pobieramy część tekstu wygenerowaną przez model i dodajemy ją do `response`.
        # `delta.content` może być `None`, więc stosujemy `or ''`, aby uniknąć błędów.
        response += chunk.choices[0].delta.content or ''

        # Używamy `yield`, co pozwala zwracać częściową odpowiedź na bieżąco.
        # Dzięki temu użytkownik widzi tekst stopniowo, zamiast czekać na pełną odpowiedź.
        yield response

In [10]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


In [9]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [10]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


History is:
[]
And messages is:
[{'role': 'system', 'content': "You are a helpful assistant in a clothes store. You should try to gently encourage the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. For example, if the customer says 'I'm looking to buy a hat', you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'Encourage the customer to buy hats if they are unsure what to get.\nIf the customer asks for shoes, you should respond that shoes are not on sale today, but remind the customer to look at hats!"}, {'role': 'user', 'content': 'co masz w ofercie'}]
History is:
[{'role': 'user', 'metadata': None, 'content': 'co masz w ofercie', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Mamy wiele wspaniałych rzeczy w ofercie! Oferujemy odzież, dodatki i wiele więcej. Szczególnie chciałbym zwrócić uwagę na nasze kapelusze, które są obecnie w promocji - aż 60% ta

In [13]:
# Fixed a bug in this function brilliantly identified by student Gabor M.!
# I've also improved the structure of this function

def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
def chat(message, history):

    # 🔹 Inicjalizujemy zmienną `relevant_system_message`, przypisując jej domyślną wiadomość systemową.
    relevant_system_message = system_message

    # 🔹 Sprawdzamy, czy w wiadomości użytkownika pojawia się słowo "belt" (pasek).
    #    Jeśli tak, dodajemy informację, że sklep nie sprzedaje pasków i sugerujemy inne produkty.
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    # 🔹 Tworzymy pełną listę wiadomości do wysłania do modelu:
    #    1. Wiadomość systemowa (`relevant_system_message`) — określa zasady rozmowy.
    #    2. Historia wcześniejszych wiadomości (`history`) — kontekst rozmowy.
    #    3. Nowa wiadomość użytkownika (`message`).
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    # 🔹 Wysyłamy dane do OpenAI:
    #    - `model=MODEL` → wskazuje, który model AI wykorzystujemy.
    #    - `messages=messages` → przekazujemy pełny kontekst rozmowy.
    #    - `stream=True` → aktywuje tryb strumieniowania (odpowiedź zwracana stopniowo).
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    # 🔹 Inicjalizujemy pusty string `response`, który będzie przechowywał odpowiedź modelu.
    response = ""

    # 🔹 Iterujemy po kolejnych fragmentach odpowiedzi zwracanych przez model.
    for chunk in stream:
        # Dodajemy nową część odpowiedzi do zmiennej `response`.
        # `delta.content` może być `None`, więc stosujemy `or ''`, aby uniknąć błędów.
        response += chunk.choices[0].delta.content or ''

        # Używamy `yield`, aby zwracać częściową odpowiedź na bieżąco.
        # Dzięki temu użytkownik widzi tekst stopniowo, co poprawia płynność interakcji.
        yield response

In [14]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business Applications</h2>
            <span style="color:#181;">Conversational Assistants are of course a hugely common use case for Gen AI, and the latest frontier models are remarkably good at nuanced conversation. And Gradio makes it easy to have a user interface. Another crucial skill we covered is how to use prompting to provide context, information and examples.
<br/><br/>
Consider how you could apply an AI Assistant to your business, and make yourself a prototype. Use the system prompt to give context on your business, and set the tone for the LLM.</span>
        </td>
    </tr>
</table>